# CP 322 - Final Project

Aditya Chauhan (169027493)

## Introduction

### Dataset Choice

- [Student Performance Dataset](https://archive.ics.uci.edu/dataset/320/student+performance)

### Key Research Questions:

- Can external factors such as familial support, alcohol consumption, absences, and many others, predict student's academic success?
- If they can, which model can most accurately use the factors to make the prediction?

### Imports

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
from scipy.stats import zscore

student_mat = pd.read_csv("Data/student-mat.csv", sep=";")
student_por = pd.read_csv("Data/student-por.csv", sep=";")

## Data Exploration

### Objective & Use Case

Student success is a cornerstone of educational systems, and identifying the drivers of academic performance is critical for improving outcomes. This project utilizes the Student Performance Dataset to develop a predictive model that analyzes the factors affecting final grades (G3). By addressing key questions—like the importance of early grades (G1, G2), the role of family support, and the impact of behavioral factors—this model provides practical tools for educators and administrators to allocate resources effectively and support at-risk students.

### Dataset Overview

In [93]:
# Identify common columns in both datasets
common_columns = set(student_mat.columns).intersection(set(student_por.columns))

# Merge the datasets on the common columns to identify overlapping students
merged_students = pd.merge(student_mat, student_por, on=list(common_columns), how='inner')

# Print the results
print(f"Number of students in student-mat dataset: {len(student_mat)}")
print(f"Number of students in student-por dataset: {len(student_por)}")
print(f"Number of students present in both datasets: {len(merged_students)}")

Number of students in student-mat dataset: 395
Number of students in student-por dataset: 649
Number of students present in both datasets: 0


### Tabular Summary

In [94]:
def generate_tabular_summary(df):
    """
    Generate a tabular summary for the dataset with key statistics.
    
    Parameters:
    - df: pandas DataFrame, the dataset to summarize.
    
    Returns:
    - summary: pandas DataFrame with feature-wise statistics.
    """
    summary = pd.DataFrame({
        "Column Name": df.columns,
        "Data Type": df.dtypes.values,
        "Number of Unique Values": df.nunique().values,
        "Number of Missing Values": df.isnull().sum().values,
        "Percentage of Missing Values (%)": (df.isnull().sum() / len(df) * 100).values,
        "Mean": [df[col].mean() if df[col].dtype in ['int64', 'float64'] else None for col in df.columns],
        "Median": [df[col].median() if df[col].dtype in ['int64', 'float64'] else None for col in df.columns],
        "Standard Deviation": [df[col].std() if df[col].dtype in ['int64', 'float64'] else None for col in df.columns],
        "Minimum": [df[col].min() if df[col].dtype in ['int64', 'float64'] else None for col in df.columns],
        "Maximum": [df[col].max() if df[col].dtype in ['int64', 'float64'] else None for col in df.columns],
        "Top Value (Mode)": [df[col].mode()[0] if not df[col].mode().empty else None for col in df.columns],
        "Frequency of Top Value": [df[col].value_counts().iloc[0] if not df[col].value_counts().empty else None for col in df.columns]
    })
    return summary

# Generate the tabular summary
tabular_summary = generate_tabular_summary(merged_data)

# Save the summary to a CSV file
tabular_summary.to_csv("tabular_summary.csv", index=False)
print("Tabular summary saved to 'tabular_summary.csv'.")

# Display the summary
tabular_summary

Tabular summary saved to 'tabular_summary.csv'.


,Column Name,Data Type,Number of Unique Values,Number of Missing Values,Percentage of Missing Values (%),Mean,Median,Standard Deviation,Minimum,Maximum,Top Value (Mode),Frequency of Top Value
0,school,object,2,0,0.0,NaN,NaN,NaN,NaN,NaN,GP,342
1,sex,object,2,0,0.0,NaN,NaN,NaN,NaN,NaN,F,198
2,age,int64,7,0,0.0,16.586387,17.0,1.173470,15.0,22.0,16,107
3,address,object,2,0,0.0,NaN,NaN,NaN,NaN,NaN,U,301
4,famsize,object,2,0,0.0,NaN,NaN,NaN,NaN,NaN,GT3,278
5,Pstatus,object,2,0,0.0,NaN,NaN,NaN,NaN,NaN,T,344
6,Medu,int64,5,0,0.0,2.806283,3.0,1.086381,0.0,4.0,4,135
7,Fedu,int64,5,0,0.0,2.565445,3.0,1.096240,0.0,4.0,2,105
8,Mjob,object,5,0,0.0,NaN,NaN,NaN,NaN,NaN,other,138
9,Fjob,object,5,0,0.0,NaN,NaN,NaN,NaN,NaN,other,211
